In [ ]:
import os
from PIL import Image

def compress_image(image_path, target_size_kb=800, step=5):
    """Compress an image to be ≤ target_size_kb (default: 250 KB)"""
    img = Image.open(image_path)
    
    # Convert PNG to JPEG for better compression
    if img.mode in ("RGBA", "P"):
        img = img.convert("RGB")
    
    # Define the output folder
    output_folder = "C:\\Users\\pc\\Desktop\\finalfolder\\data\\processed"
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

    # Create output path with the same filename in the processed folder
    output_path = os.path.join(output_folder, os.path.basename(image_path))
    
    quality = 99  # Start with high quality

    while quality > 10:
        img.save(output_path, "JPEG", quality=quality, optimize=True)
        file_size_kb = os.path.getsize(output_path) / 1024  # Convert bytes to KB

        if file_size_kb >= target_size_kb:
            print(f"✔ {os.path.basename(image_path)} compressed to {file_size_kb:.2f} KB at quality {quality}")
            return
        quality -= step  # Reduce quality step by step

    print(f"⚠ {os.path.basename(image_path)} could not be reduced below {file_size_kb:.2f} KB")

def compress_all_images(folder_path, target_size_kb=800):
    """Compress all images in a folder to ≤ target_size_kb (default: 150 KB)"""
    if not os.path.exists(folder_path):
        print("❌ Folder does not exist!")
        return

    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(root, file)
                compress_image(image_path, target_size_kb)

    print("\n✅ All images compressed successfully!")

# Example usage: Specify the folder where images are located
compress_all_images(r"C:\\Users\\pc\\Desktop\\finalfolder\\data\\raw\\Upload Image(passport size) (File responses)")  # Provide the correct folder path here


✔ 21ESBCA003.jpg compressed to 440.69 KB at quality 99
✔ 21ESBCA004.jpg compressed to 143.73 KB at quality 99
✔ 21ESBCA005.jpg compressed to 50.49 KB at quality 99
✔ 21ESBCA007.jpg compressed to 313.85 KB at quality 99
✔ 21ESBCA008.jpg compressed to 317.84 KB at quality 99
✔ 21ESBCA009.jpg compressed to 325.67 KB at quality 99
✔ 21ESBCA011.jpg compressed to 366.04 KB at quality 99
✔ 21ESBCA013.jpg compressed to 442.72 KB at quality 99
✔ 21ESBCS002.jpg compressed to 64.41 KB at quality 99
✔ 21ESBCS003.jpg compressed to 201.19 KB at quality 99
✔ 21esbcs006.jpg compressed to 107.41 KB at quality 99
✔ 21ESBCS007.jpg compressed to 209.35 KB at quality 99
✔ 21ESBCS009.jpg compressed to 175.35 KB at quality 99
✔ 21ESBCS010.jpg compressed to 219.59 KB at quality 99
✔ 21ESBCS015.jpg compressed to 327.41 KB at quality 99
✔ 21ESBCS016.jpg compressed to 312.31 KB at quality 99
✔ 21ESBCS018.jpg compressed to 297.04 KB at quality 99
✔ 21ESBCS019.jpg compressed to 139.50 KB at quality 99
✔ 21ESBCS021

In [11]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# Input and output folder paths
input_folder = "C:\\Users\\pc\\Desktop\\faceattendencemodel\\data\\raw\\Upload Image(passport size) (File responses)"
output_base_folder = "C:\\Users\\pc\\Desktop\\faceattendencemodel\\data\\augmented_faces"

# Create output folder if it doesn't exist
os.makedirs(output_base_folder, exist_ok=True)

# Data Augmentation settings
datagen = ImageDataGenerator(
    rotation_range=30,      
    width_shift_range=0.2,  
    height_shift_range=0.2, 
    shear_range=0.3,        
    zoom_range=0.3,         
    horizontal_flip=True,   
    brightness_range=[0.8, 1.2],  
    fill_mode="nearest"
)

# Process each image in the input folder
for img_name in os.listdir(input_folder):
    if img_name.endswith((".jpg", ".png")):
        person_name = os.path.splitext(img_name)[0].strip()  # Remove spaces
        img_path = os.path.join(input_folder, img_name)

        # Load and resize image
        img = Image.open(img_path)
        img = img.convert("RGB")  # Convert to RGB to avoid grayscale issues
        img = img.resize((160, 160))  # Resize to match face recognition models

        # Create a separate folder for each person
        person_folder = os.path.join(output_base_folder, person_name.replace(" ", "_"))
        os.makedirs(person_folder, exist_ok=True)

        # Convert image to numpy array
        img_array = np.array(img)

        # Ensure it has 3 color channels
        if img_array.ndim == 3 and img_array.shape[-1] != 3:
            img_array = np.stack((img_array,) * 3, axis=-1)  # Convert grayscale to RGB

        # Add batch dimension (batch_size=1)
        img_array = np.expand_dims(img_array, axis=0)

        # Debugging: Print shape to verify
        print(f"Processing {img_name} - Final shape: {img_array.shape}")  # Should be (1, 160, 160, 3)

        # Generate and save augmented images
        num_generated = 10  # Number of images per person
        i = 0
        for batch in datagen.flow(img_array, batch_size=1, save_to_dir=person_folder, 
                                  save_prefix=person_name.replace(" ", "_"), save_format="jpg"):
            i += 1
            if i >= num_generated:
                break  # Stop after generating required images

        print(f"✅ {num_generated} images saved for {person_name} in '{person_folder}/' 🎉")

print("\n✅ Augmentation completed for all persons!")


Processing 21ESBCA003.jpg - Final shape: (1, 160, 160, 3)
✅ 10 images saved for 21ESBCA003 in 'C:\Users\pc\Desktop\faceattendencemodel\data\augmented_faces\21ESBCA003/' 🎉
Processing 21ESBCA004.jpg - Final shape: (1, 160, 160, 3)
✅ 10 images saved for 21ESBCA004 in 'C:\Users\pc\Desktop\faceattendencemodel\data\augmented_faces\21ESBCA004/' 🎉
Processing 21ESBCA005.jpg - Final shape: (1, 160, 160, 3)
✅ 10 images saved for 21ESBCA005 in 'C:\Users\pc\Desktop\faceattendencemodel\data\augmented_faces\21ESBCA005/' 🎉
Processing 21ESBCA007.jpg - Final shape: (1, 160, 160, 3)
✅ 10 images saved for 21ESBCA007 in 'C:\Users\pc\Desktop\faceattendencemodel\data\augmented_faces\21ESBCA007/' 🎉
Processing 21ESBCA008.jpg - Final shape: (1, 160, 160, 3)
✅ 10 images saved for 21ESBCA008 in 'C:\Users\pc\Desktop\faceattendencemodel\data\augmented_faces\21ESBCA008/' 🎉
Processing 21ESBCA009.jpg - Final shape: (1, 160, 160, 3)
✅ 10 images saved for 21ESBCA009 in 'C:\Users\pc\Desktop\faceattendencemodel\data\augme

In [12]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from mtcnn import MTCNN

class FACELOADING:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (256, 256)  # More details, better quality
        self.X = []
        self.Y = []
        self.person_names = []
        self.detector = MTCNN()

    def extract_face(self, filename):
        img = cv.imread(filename)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        faces = self.detector.detect_faces(img)
        if faces:
            x, y, w, h = faces[0]['box']
            x, y = abs(x), abs(y)
            face = img[y:y+h, x:x+w]
            face_arr = cv.resize(face, self.target_size, interpolation=cv.INTER_CUBIC)
            return face_arr
        else:
            return None

    def load_faces(self, dir):
        FACES = []
        for im_name in os.listdir(dir):
            try:
                path = os.path.join(dir, im_name)
                single_face = self.extract_face(path)
                if single_face is not None:
                    FACES.append(single_face)
            except Exception as e:
                pass
        return FACES

    def load_classes(self):
        for sub_dir in os.listdir(self.directory):
            path = os.path.join(self.directory, sub_dir)
            if os.path.isdir(path):
                FACES = self.load_faces(path)
                labels = [sub_dir for _ in range(len(FACES))]
                print(f"Loaded successfully: {len(labels)}")
                self.X.extend(FACES)
                self.Y.extend(labels)
                self.person_names.extend([sub_dir for _ in range(len(FACES))])  # Store the person name (folder name)
        
        return np.asarray(self.X), np.asarray(self.Y), self.person_names

    def save_faces(self, save_directory):
        # Check if the directory exists, if not create it
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)

        for i, image in enumerate(self.X):
            label = self.Y[i]
            label_dir = os.path.join(save_directory, label)

            # Create a folder for each label if not exists
            if not os.path.exists(label_dir):
                os.makedirs(label_dir)

            # Construct the path to save the image
            save_path = os.path.join(label_dir, f"face_{i+1}.jpg")
            # Convert RGB to BGR for OpenCV compatibility before saving
            image_bgr = cv.cvtColor(image, cv.COLOR_RGB2BGR)
            cv.imwrite(save_path, image_bgr)
            print(f"Saved: {save_path}")

# Example usage:
faceloading = FACELOADING("C:\\Users\\pc\\Desktop\\faceattendencemodel\\data\\augmented_faces")
X, Y, Z = faceloading.load_classes()
faceloading.save_faces("C:\\Users\\pc\\Desktop\\faceattendencemodel\\data\\train")


Loaded successfully: 28
Loaded successfully: 30
Loaded successfully: 30
Loaded successfully: 30
Loaded successfully: 30
Loaded successfully: 28
Loaded successfully: 30
Loaded successfully: 27
Loaded successfully: 28
Loaded successfully: 30
Loaded successfully: 28
Loaded successfully: 30
Loaded successfully: 25
Loaded successfully: 25
Loaded successfully: 29
Loaded successfully: 26
Loaded successfully: 30
Loaded successfully: 30
Loaded successfully: 29
Loaded successfully: 29
Loaded successfully: 28
Loaded successfully: 28
Loaded successfully: 29
Loaded successfully: 30
Loaded successfully: 25
Loaded successfully: 30
Loaded successfully: 30
Loaded successfully: 29
Loaded successfully: 21
Loaded successfully: 30
Loaded successfully: 30
Loaded successfully: 30
Loaded successfully: 30
Loaded successfully: 24
Loaded successfully: 30
Loaded successfully: 30
Loaded successfully: 6
Loaded successfully: 30
Loaded successfully: 14
Loaded successfully: 27
Saved: C:\Users\pc\Desktop\faceattendence

In [13]:
from keras_facenet import FaceNet
embedder = FaceNet()

def get_embedding(face_img):
    face_img = face_img.astype('float32') # 3D(160x160x3)
    face_img = np.expand_dims(face_img, axis=0) 
    # 4D (Nonex160x160x3)
    yhat= embedder.embeddings(face_img)
    return yhat[0] # 512D image (1x1x512)

EMBEDDED_X = []

for img in X:
    EMBEDDED_X.append(get_embedding(img))

EMBEDDED_X = np.asarray(EMBEDDED_X)
e=input("enter the passing year")
np.savez_compressed(f'C:\\Users\\pc\\Desktop\\faceattendencemodel\\models\\{e}.npz', EMBEDDED_X=EMBEDDED_X, Y=Y, Z=Z)



1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━